# 🗓️ Calendar Function Calling Tutorial with Friendli Suite

In [ ]:
%pip install -r requirements.txt

### Setup environment

1. Preparing Friendli Personal Access Token:  
   a. Sign up at [Friendli Suite](https://suite.friendli.ai).  
   b. Get a [Personal Access Token (PAT)](https://suite.friendli.ai/user-settings/tokens)  
   c. Copy the token, it will be used in the following steps.  

2. Setup the Google Cloud Platform Credentials for google calendar API:  
   a. Create a project in the [Google Cloud Console](https://developers.google.com/workspace/guides/create-project)  
   b. Follow the steps in the [Google Calendar API Python Quickstart](https://developers.google.com/calendar/api/quickstart/python#set_up_your_environment) to enable the Google Calendar API and download the credentials.json file.  
   c. rename the `credentials-xxxxx.json` file to `credentials.json` and upload it to the current working directory.

In [ ]:
# Step 1: Set up Friendli Personal Access Token and Google Calendar API client
from datetime import datetime, timedelta
from getpass import getpass
from typing import Annotated
import typer
import os

from gcsa.google_calendar import GoogleCalendar
from gcsa.event import Event


if not os.environ.get("FRIENDLI_TOKEN"):
    os.environ["FRIENDLI_TOKEN"] = getpass.getpass("Enter your Friendli Token: ")

if not os.environ.get("GMAIL"):
    os.environ["GMAIL"] = input("Enter your gmail: ")

fai_token = os.environ.get("FRIENDLI_TOKEN")
gmail = os.environ.get("GMAIL")

gc = GoogleCalendar(gmail, credentials_path='credentials.json')

In [ ]:
# Step 2: Define calendar event tools
def get_events_for_period(
    start_date: Annotated[str, 'Start Date of the event, format: yyyy-mm-dd'], 
    end_date: Annotated[str, 'End date of the event, format: yyyy-mm-dd']
):
    """Selects two dates and returns calendar events between those dates."""
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    calendar = gc.get_events(
        time_min=start,
        time_max=end + timedelta(days=1) - timedelta(seconds=1),
        order_by='startTime',
        single_events=True,
    )
    
    sd, ed = start.date(), end.date()
    header = f"==== {sd if sd == ed else f'{sd} ~ {ed}'} events ====\n\n"
    result = ""
    for event in calendar:
        result += f"summary: {event.summary}, date: {event.start}{event.start != event.end and f' ~ {event.end}'}\n"
        result += "-----------------------------------\n"
    result = result or "No events found for the given date range."
    return header + result


def get_events_for_date(select_date: Annotated[str, 'Date of the event, format: yyyy-mm-dd']):
    """Selects a date and returns calendar events for that date."""
    return get_events_for_period(select_date, select_date)


def get_events_for_week(point_date: Annotated[str, 'a included date in the week, format: yyyy-mm-dd']):
    """Returns all schedules for the week containing the entered date."""
    point_date_obj = datetime.strptime(point_date, "%Y-%m-%d")
    day_of_week = point_date_obj.weekday()
    one_week_later = point_date_obj + timedelta(days=6 - day_of_week)
    one_week_earlier = point_date_obj - timedelta(days=day_of_week) 

    return get_events_for_period(one_week_earlier.strftime("%Y-%m-%d"), one_week_later.strftime("%Y-%m-%d"))


def get_events_by_query(query: Annotated[str, 'Search using key words and text of the event you are looking for']) -> None:
    """Searches for events using keywords and text."""
    calendar = gc.get_events(
        q=query,
        order_by='startTime',
        single_events=True,
    )
    
    result = ""
    for event in calendar: 
        result += f"summary: {event.summary} || date:{ event.start} {event.start != event.end and f'~ {event.end}'}\n"
        result += "-----------------------------------\n"
    return result or "No events found"


def add_event_for_date(
    date: Annotated[str, 'Date of the event, format: yyyy-mm-dd'],
    summary: Annotated[str, 'Summary of the event']
):
    """Adds a new event to the calendar with a given date and summary"""
    parse = datetime.strptime(date, "%Y-%m-%d").date()
    print(f"Event summary: {summary} - {parse}")


    typer.secho(f"Are you sure you want to add this event? (Y/n): ", fg=typer.colors.YELLOW)
    confirm = input("(Y / n): ")
    if confirm not in ("y", "Y", ""):
        return "Event addition canceled"

    event = Event(summary="[AI] " + summary, start=parse, description="\n\n\nEvent added by FriendliAI", color_id='7')
    gc.add_event(event)
    return "Event added successfully"

In [ ]:
# Step 3: Define web search tool

from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

def web_search(
    query: Annotated[str, 'Search query'], 
    max_results: Annotated[int, 'Max number of results to return'] = 5
):
    """A wrapper around DuckDuckGo Search API. Useful for when you need to answer a question or find information. Input should be a search query."""
    return search.run(query, max_results)


In [ ]:
# Step 4: Define tool set

from function_schema import get_function_schema


def get_tool_schema(function):
    return {
        "type": "function",
        "function": get_function_schema(function)
    }

available_tools = {
    "web_search": web_search,
    "add_event_for_date": add_event_for_date,
    "get_events_for_week": get_events_for_week,
    "get_events_for_date": get_events_for_date,
    "get_events_for_period": get_events_for_period,
    "get_events_by_query": get_events_by_query,
}

tools = [
    get_tool_schema(tool) for tool in available_tools.values()
]

In [ ]:
# Step 5: Define the Friendli agent

import json
from datetime import datetime

from friendli import Friendli


def run_agent():
    today = datetime.now().strftime('%Y-%m-%d')
    messages = [
        {
            "role": "system",
            "content": f"\nYou are a helpful assistant.\nToday is {today}."
        },
    ]

    # Initialize the client
    client = Friendli(token=fai_token)

    # Continuous loop that processes user inputs and model responses
    while True:
        # Prompt the user for input if the last message is not from a tool
        if messages[-1]["role"] != "tool":
            user_query = typer.prompt("User", default="bye", show_default=False)
            if user_query in ("bye", ""):
                break

            messages.append({
                "role": "user",
                "content": user_query,
            })
            typer.secho(f" {user_query}")

        # Generate a response from the model
        resp = client.chat.completions.create(
            
            model="meta-llama-3.1-8b-instruct",
            messages=messages,
            tools=tools,
        )
        resp_content = resp.choices[0].message.content
        resp_tool_calls = resp.choices[0].message.tool_calls

        # Prepare the assistant's message
        assistant_msg = {
            "role": "assistant"
        }
        # If there is content in the response, display it and add it to the assistant's message
        if resp_content:
            typer.secho(f"\nFriendliAI: {resp_content}", fg=typer.colors.BLUE)
            assistant_msg["content"] = resp_content

        # List to hold tool response messages
        tool_messages = []

        if resp_tool_calls:
            # If there is tool_calls in the response, add them to the assistant's message
            assistant_msg["tool_calls"] = resp_tool_calls
            # Iterate through each tool call and execute the corresponding function
            typer.secho(f"TOOL CALLS: {len(resp_tool_calls)}", fg=typer.colors.YELLOW)

            for tool_call in resp_tool_calls:
                typer.secho(f"TOOL CALLS: {tool_call.function.name}({tool_call.function.arguments})", fg=typer.colors.YELLOW)

            for tool_call in resp_tool_calls:
                tool = available_tools[tool_call.function.name]
                call_args = json.loads(tool_call.function.arguments)
                try:
                    result = tool(**call_args)
                except Exception as e:
                    result = f"An error occurred: {e}"
                    typer.secho(f"TOOL CALL ERROR: {result}", fg=typer.colors.RED)

                tool_messages.append({
                    "role": "tool",
                    "content": result,
                    "tool_call_id": tool_call.id,
                })

        messages.append(assistant_msg)
        if tool_messages:
            messages.extend(tool_messages)


In [ ]:
run_agent()